## Utilizando Web-Scraping para fazer pesquisa de preço

#### Passo 1: instalar bibliotecas

In [74]:
# Remova o comentário(#) da linha abaixo caso esteja executando este script pela primeira vez
!pip install requests
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


#### Passo 2: Importar bibliotecas

In [75]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Passo 3: Formatar link de busca

In [76]:
# 1) Copiar o modelo de link de busca
url_de_busca = 'https://lista.mercadolivre.com.br/'

# 2) Informar a palavra chave para busca
palavra_chave = 'smart tv 70'

# 3) Formatar a palavra chave conforme modelo deste site
palavra_chave = palavra_chave.replace(' ','-')

# 4) Formar o link
link = url_de_busca+palavra_chave
print(link)

https://lista.mercadolivre.com.br/smart-tv-70


#### Passo 4: Obter informações do link

In [77]:
#Requisita informações do site
response = requests.get(link)

# Verifica status de acesso
print('Código de status', response.status_code)

Código de status 403


Esse **403 Forbidden** acontece porque o Mercado Livre (assim como outros grandes e-commerces) possui mecanismos de proteção contra bots e scrapers simples, e o requests.get() sem nenhum cabeçalho já denuncia que não é um navegador “de verdade”.

#### 4.1 Usar um User-Agent válido

Alguns sites **bloqueiam requisições que não parecem vir de navegadores**. Então você pode **enviar cabeçalhos simulando um browser**.

##### Para criar um User-Agent (real)
- Identifique qual é a versão do seu navegador acessando, por exemplo, a URL chrome://settings/help
- Crie um cabeçalho com a versão do seu navegador.

In [78]:
#Cabeçalho com User-Agent
headers = {'User-Agent': 'Chrome/139.0.7258.139'}

#Requisita informações do site
response = requests.get(link, headers = headers)


#Verifica status de acesso
print('Código de status:', response.status_code)

Código de status: 403


In [79]:
# Cria um objeto BeautifulSoup com as informações html
soup = BeautifulSoup(response.content, 'html.parser')

#### Passo 5: Encontrar elementos

In [82]:
resultados = soup.find_all(class_ = 'ui-search-layout__item')

len(resultados)

0

#### Passo 6: Visualizar links dos produtos

In [50]:
# 1) Criar lista vazia para armazenar links 
lista_de_links = []

# 2) Criar laço de repetição para filtrar todos os dados
for i in resultados:
    lista_de_links.append(i.find('a')['href'])

# 3) Visualizar Links
for j in lista_de_links:
    print(j)

https://www.mercadolivre.com.br/smart-tv-58-philco-led-4k-google-tv-hdr10-p58kga/p/MLB48895726#polycard_client=search-nordic&searchVariation=MLB48895726&wid=MLB4060862571&position=3&search_layout=grid&type=product&tracking_id=c0c26f4f-0a2a-422a-aae5-3c1fe8ce6fd1&sid=search
https://www.mercadolivre.com.br/smart-tv-70-crystal-uhd-4k-70du8000-2024-samsung/p/MLB35650763#polycard_client=search-nordic&searchVariation=MLB35650763&wid=MLB4660135102&position=4&search_layout=grid&type=product&tracking_id=c0c26f4f-0a2a-422a-aae5-3c1fe8ce6fd1&sid=search
https://www.mercadolivre.com.br/smart-tv-50-philco-ptv50va4regb-roku-tv-led-dolby-audio/p/MLB44408782#polycard_client=search-nordic&searchVariation=MLB44408782&wid=MLB3972474057&position=5&search_layout=grid&type=product&tracking_id=c0c26f4f-0a2a-422a-aae5-3c1fe8ce6fd1&sid=search
https://www.mercadolivre.com.br/smart-tv-aoc-roku-led-50-4k-50u612578g-hdmiusb-preto/up/MLBU3044199534#polycard_client=search-nordic&searchVariation=MLBU3044199534&wid=MLB

#### Passo 7: Extrarir informações de cada produto listado

In [51]:
produto = BeautifulSoup(requests.get(lista_de_links[0], headers = headers).content, 'html.parser')

In [52]:
# Selecionar o título
produto.find(class_ = 'ui-pdp-title').text

'Smart Tv 58 Philco Led 4k Google Tv Hdr10 P58kga'

In [55]:
# Selecionar Preço
produto.find(class_ = 'andes-money-amount__fraction').text

'3.284'

In [56]:
# Selecionar a quantidade de vendas
produto.find(class_ = 'ui-pdp-subtitle').text

'Novo  |  +5mil vendidos'

In [58]:
# Selecionar a descrição
produto.find(class_ = 'ui-pdp-description__content').text

'A Smart TV Philco 58” 4K UHD P58KGA com Google TV foi desenvolvida para quem busca aliar alta qualidade de imagem, conectividade inteligente e excelente custo-benefício. Com tela de 58 polegadas, resolução Ultra HD e suporte a HDR10, ela proporciona uma experiência visual rica em detalhes, cores vibrantes e contraste superior — ideal para filmes, séries, esportes e games. Seu design moderno com bordas finas confere sofisticação ao ambiente, seja na sala de estar, quarto ou até em espaços comerciais.Equipada com o sistema operacional Google TV, a P58KGA oferece uma navegação fluida, intuitiva e altamente personalizada. O acesso direto a aplicativos como Netflix, YouTube, Prime Video e Globoplay, além de milhares de outros via Google Play, transforma a televisão em um verdadeiro hub de entretenimento. Com controle remoto por voz, comandos integrados ao Google Assistente e Chromecast embutido, o usuário tem liberdade total para interagir com a TV sem tocar em nenhum botão.Outro destaque 

#### Criar tabela para armazenar dados

In [59]:
# 1) Criar o nome de cada coluna
colunas = ['Título','Preço','Quantidade de vendas','Descrição do produto']

# 2) Criar a tabela com as colunas informadas
tabela = pd.DataFrame(columns = colunas)

# 3) Visualizar tabela
tabela

,Título,Preço,Quantidade de vendas,Descrição do produto


In [65]:
# Criar laço de repetição para armazenar as informações de todos os produtos da lista de links

id = 0 # Contador
for j in lista_de_links:
    try:
        #Requisita informações do link do produto e cria o objeto soup
        produto = BeautifulSoup(requests.get(j, headers = headers).content, 'html.parser')
        # Selecionar o título
        titulo = produto.find(class_ = 'ui-pdp-title').text
        # Selecionar Preço
        preco = produto.find(class_ = 'andes-money-amount__fraction').text
        # Selecionar a quantidade de vendas
        vendidos = produto.find(class_ = 'ui-pdp-subtitle').text
        # Selecionar a descrição
        descricao = produto.find(class_ = 'ui-pdp-description__content').text

        # Inserir uma nova linha na tabela com as informações do produto
        tabela.loc[len(tabela)] = [titulo,preco,vendidos,descricao]
    except:
        print(f"Houve um problema no link de índice {id}")

    # Incrementa uma posição no id
    id = id + 1

Houve um problema no link de índice 0
Houve um problema no link de índice 1
Houve um problema no link de índice 2
Houve um problema no link de índice 3
Houve um problema no link de índice 4
Houve um problema no link de índice 5
Houve um problema no link de índice 6
Houve um problema no link de índice 7
Houve um problema no link de índice 8
Houve um problema no link de índice 9
Houve um problema no link de índice 10
Houve um problema no link de índice 11
Houve um problema no link de índice 12
Houve um problema no link de índice 13
Houve um problema no link de índice 14
Houve um problema no link de índice 15
Houve um problema no link de índice 16
Houve um problema no link de índice 17
Houve um problema no link de índice 18
Houve um problema no link de índice 19
Houve um problema no link de índice 20
Houve um problema no link de índice 21
Houve um problema no link de índice 22
Houve um problema no link de índice 23
Houve um problema no link de índice 24


In [66]:
tabela

,Título,Preço,Quantidade de vendas,Descrição do produto


#### Passo 8: Salvar tabela em uma planilha de Excel

### Exercício 1: Encontrar uma solução para listar todos os resultados quando houver 2 ou mais páginas de resultado

### Exercício 1.1 (extra): Formatar os dados da tabela

### Exercício 2: Faça o mesmo processo para outro marketplace, por exemplo, Magazine Luiza, Amazon, etc.